In [110]:
!pip install pandas xlsxwriter


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [111]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()

In [112]:
spark.sql("SHOW TABLES IN demo.nyc").show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|      nyc|           taxis_100|      false|
|      nyc|     taxis_100M_time|      false|
|      nyc|taxis_1000_50COLUMNS|      false|
|      nyc|            taxis_1B|      false|
|      nyc|taxis_100_50COLUM...|      false|
|      nyc|taxis_100000_50CO...|      false|
|      nyc|            taxis_1L|      false|
|      nyc|          taxis_1000|      false|
|      nyc|            taxis_10|      false|
|      nyc| taxis_10M_50COLUMNS|      false|
|      nyc|   taxis_1L_5COLUMNS|      false|
|      nyc|         taxis_10000|      false|
|      nyc|            taxis_1M|      false|
|      nyc|          taxis_1L_5|      false|
|      nyc|          taxis_10_M|      false|
|      nyc|taxis_1000_50COLU...|      false|
|      nyc|  taxis_10_50COLUMNS|      false|
|      nyc|           taxis_10K|      false|
|      nyc|            taxis_1K|      false|
|      nyc

In [114]:
# spark.sql("SHOW TABLES IN demo").show()


In [116]:
# spark.sql("REFRESH TABLE demo.nyc.taxis_100_50COLUMNS")


In [119]:
import time
from pyspark.sql.functions import col, when, lit, rand
from pyspark.sql import functions as F

# Load the existing Apache Iceberg table
df = spark.read.format("iceberg").load("demo.nyc.taxis_1000_50COLUMNS")

# Get total record count
total_rows = df.count()
print(f"Total rows in Iceberg table: {total_rows}")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = int(total_rows * update_percentage)

print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample unique IDs to update
sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

st = time.time()

# Update records using Iceberg's MERGE INTO
query = f"""
MERGE INTO demo.nyc.taxis_1000_50COLUMNS AS target
USING (
    SELECT extra_col_0 FROM demo.nyc.taxis_1000_50COLUMNS
    WHERE extra_col_0 IN ({",".join(map(str, sampled_ids))})
) AS source
ON target.extra_col_0 = source.extra_col_0
WHEN MATCHED THEN 
    UPDATE SET target.extra_col_1 = target.extra_col_1 + 10
"""
spark.sql(query)

end = time.time() - st
print(f"Updated {num_rows} rows in {end:.2f} sec")


Py4JJavaError: An error occurred while calling o167724.load.
: org.apache.iceberg.exceptions.NotFoundException: Location does not exist: s3://warehouse/nyc/taxis_1M_50COLUMNS/metadata/00001-cc09381d-366c-4903-9bd4-ee42e78d4857.metadata.json
	at org.apache.iceberg.aws.s3.S3InputStream.openStream(S3InputStream.java:194)
	at org.apache.iceberg.aws.s3.S3InputStream.positionStream(S3InputStream.java:177)
	at org.apache.iceberg.aws.s3.S3InputStream.read(S3InputStream.java:107)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.ensureLoaded(ByteSourceJsonBootstrapper.java:539)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.detectEncoding(ByteSourceJsonBootstrapper.java:133)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.constructParser(ByteSourceJsonBootstrapper.java:256)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.JsonFactory._createParser(JsonFactory.java:17

In [108]:
import time
from pyspark.sql import SparkSession

# Initialize Spark Session
# spark = SparkSession.builder \
#     .appName("Iceberg Merge") \
#     .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
#     .config("spark.sql.catalog.demo.type", "hadoop") \
#     .config("spark.sql.catalog.demo.warehouse", "s3://your-bucket/path-to-iceberg/") \
#     .getOrCreate()

# Load the existing Apache Iceberg table
df = spark.read.format("iceberg").load("demo.nyc.taxis_1000_50COLUMNS")

# Get total record count
total_rows = df.count()
print(f"Total rows in Iceberg table: {total_rows}")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# ✅ Use a deterministic sampling approach (e.g., modulo filtering)
sampled_df = df.filter("extra_col_0 % 10 = 0").limit(num_rows)

# Record start time
st = time.time()

# Create a temporary table for sampled records
sampled_df.createOrReplaceTempView("sampled_ids")

# ✅ Perform deterministic Iceberg MERGE INTO
query = """
MERGE INTO demo.nyc.taxis_1000_50COLUMNS AS target
USING sampled_ids AS source
ON target.extra_col_0 = source.extra_col_0
WHEN MATCHED THEN 
    UPDATE SET target.extra_col_1 = target.extra_col_1 + 10
"""
spark.sql(query)

# Print execution time
end = time.time() - st
print(f"Updated {num_rows} rows in {end:.2f} sec")


Py4JJavaError: An error occurred while calling o167698.load.
: org.apache.iceberg.exceptions.NotFoundException: Location does not exist: s3://warehouse/nyc/taxis_1000_50COLUMNS/metadata/00046-77d1c2f6-2413-489f-bc2b-08d524a7594a.metadata.json
	at org.apache.iceberg.aws.s3.S3InputStream.openStream(S3InputStream.java:194)
	at org.apache.iceberg.aws.s3.S3InputStream.positionStream(S3InputStream.java:177)
	at org.apache.iceberg.aws.s3.S3InputStream.read(S3InputStream.java:107)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.ensureLoaded(ByteSourceJsonBootstrapper.java:539)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.detectEncoding(ByteSourceJsonBootstrapper.java:133)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.constructParser(ByteSourceJsonBootstrapper.java:256)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.JsonFactory._createParser(JsonFactory.java:

In [55]:
# spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_1M_50COLUMNS")


DataFrame[]

In [56]:
# import os
# iceberg_table_dir = "../warehouse/nyc/taxis_1M_50COLUMNS_Update_5"
# metadata_dir = f"{iceberg_table_dir}/metadata"
# data_dir = f"{iceberg_table_dir}/data"
# input_data_dir = f"../input_data"
# analysis_info = []
# records_before_op = 0

# def append_to_file(file_path, msg):
#     open_mode = "a"
#     if not os.path.exists(file_path):
#         open_mode = "w"

#     # Open the CSV file in write mode
#     with open(file_path, open_mode) as file:
#         writer = csv.writer(file)
        
#         if open_mode=="w":
#             #writing header of the columns
#             writer.writerows([list(msg.keys())])    

#         row_values = [list(msg.values())]
#         # Write the data to the CSV file
#         writer.writerows(row_values)

# def get_size():
#     # List the metadata files
#     manifest_pattern = re.compile(r".*-m\d+\.avro$")
#     metadata_files = os.listdir(metadata_dir)
    
#     # Initialize variables to store the sizes of different types of metadata files
#     snap_avro_size = 0
#     metadata_json_size = 0
#     m_avro_size = 0

#     data_dir_size = 0
#     # get data dir size
#     data_dir_files = os.listdir(data_dir)
#     # print(data_dir_files)
#     for filename in data_dir_files:
#         file_path = os.path.join(data_dir, filename)
#         data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
#     # Iterate through the metadata files and calculate their sizes
#     for file in metadata_files:
#         file_path = os.path.join(metadata_dir, file)
#         file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
#         if file.startswith("snap-") and file.endswith(".avro"):
#             snap_avro_size += file_size_kb
#         elif file.endswith(".metadata.json"):
#             metadata_json_size += file_size_kb
#         elif manifest_pattern.match(file):
#             m_avro_size += file_size_kb
    
#     # Print the time taken and the sizes of the metadata files
#     # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
#     # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
#     # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
#     # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

#     return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [57]:
# from pyspark.sql.types import (
#     DoubleType, FloatType, LongType, StructType, StructField, 
#     StringType, IntegerType, DateType
# )

# # Define the schema with 50 columns based on the required data types
# schema = StructType([
#     # StructField("vendor_id", LongType(), True),  # INT
#     # StructField("trip_id", LongType(), True),  # INT
#     # StructField("trip_distance", FloatType(), True),  # FLOAT
#     # StructField("fare_amount", DoubleType(), True),  # DOUBLE
#     # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# # ] + [
#     # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
#     StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
#     StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
#     StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
#     StructField(f"extra_col_{i}", DateType(), True)  # DATE
#     for i in range(50)
# ])

# # Create an empty DataFrame with the schema
# df = spark.createDataFrame([], schema)

# # Create the Iceberg table
# df.writeTo("demo.nyc.taxis_1M_50COLUMNS_Update_5").create()


In [58]:
# df = spark.table("demo.nyc.taxis_1M_50COLUMNS_Update_5")
# df.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [59]:
# import time, csv
# from pyspark.sql.functions import col, when, lit
# from pyspark.sql import functions as F
# import os

# input_data_dir = "../input_data"
# output_dir = "../output"
# analysis_info = []
# records_before_op = 0
# total_insertion_time = 0

# file_type = input("Enter input file type csv or parquet? : ").lower().strip()
# input_data_dir = os.path.join(input_data_dir, file_type)
# input_files = os.listdir(input_data_dir)

# analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
# if os.path.exists(analysis_file):
#     os.remove(analysis_file)

# df = spark.table("demo.nyc.taxis_1M_50COLUMNS_Update_5")
# records_before_op = df.count()

# for file in input_files:
#     print(f"Started with file={file}")
#     file_path = os.path.join(input_data_dir, file)

#     st = time.time()
#     if file_type == "parquet":
#         df = spark.read.parquet(file_path)
#     else:
#         df = spark.read.csv(file_path, header=True)
#         df = df.select(
#             F.col("extra_col_0").cast("long"),
#             F.col("extra_col_1").cast("int"),
#             F.col("extra_col_2").cast("string"),
#             F.col("extra_col_3").cast("date"),
#             *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
#         )

#     rows = df.count()
    
#     df.writeTo("demo.nyc.taxis_1M_50COLUMNS_Update_5").append()
#     end = time.time() - st
#     total_insertion_time += end

#     details = {"time_taken": f"{end:.2f} sec", "Operation": f"Inserted {rows} records", "records_after_op": records_before_op + rows}
#     records_before_op += rows

#     print(f"Inserted {rows} records in {end:.2f} sec.")

# # **PRINT INSERTION TIME BEFORE UPDATE**
# print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# # === Get Update Percentage from User ===
# update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

# df = spark.table("demo.nyc.taxis_1M_50COLUMNS_Update_5")
# total_rows = df.count()

# num_rows = int(total_rows * update_percentage)
# print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# # Sample random rows ensuring unique IDs
# sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
# sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

# st = time.time()

# # Updating `extra_col_1` (integer)
# updated_df = df.withColumn(
#     "extra_col_1",
#     when(col("extra_col_0").isin(sampled_ids), col("extra_col_1") + 10)
#     .otherwise(col("extra_col_1"))
# )

# # Updating `extra_col_3` (date) - Commented out
# # updated_df = updated_df.withColumn(
# #     "extra_col_3",
# #     when(col("extra_col_0").isin(sampled_ids), F.date_add(col("extra_col_3"), 5))
# #     .otherwise(col("extra_col_3"))
# # )

# # Overwrite updated data
# updated_df.writeTo("demo.nyc.taxis_1M_50COLUMNS_Update_5").overwritePartitions()

# end = time.time() - st

# print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_1_1740397925.6984496.parquet


Inserted 1000000 records in 33.17 sec.

Total insertion time: 33.17 sec



Enter update percentage (e.g., 1 for 1%):  10


Updating 100000 rows (~10.0%)...


25/02/26 13:44:29 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


Updated 100000 rows in 154.69 sec


Total rows in Iceberg table: 100000


Enter update percentage (e.g., 1 for 1%):  1


Updating 1000 rows (~1.0%)...


Updated 1000 rows in 3.37 sec


In [118]:
# import time
# from pyspark.sql.functions import col, rand

# # Load the existing Apache Iceberg table
# df = spark.read.format("iceberg").load("demo.nyc.taxis_100000_50COLUMNS")

# # Get total record count
# total_rows = df.count()
# print(f"Total rows in Iceberg table: {total_rows}")

# # === Get Update Percentage from User ===
# update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
# num_rows = int(total_rows * update_percentage)
# print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# # Sample unique IDs to update
# sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(num_rows)

# st = time.time()

# # Create a temporary table for sampled records
# sampled_df.createOrReplaceTempView("sampled_ids")

# # Perform Iceberg MERGE INTO
# query = """
# MERGE INTO demo.nyc.taxis_100000_50COLUMNS AS target
# USING sampled_ids AS source
# ON target.extra_col_0 = source.extra_col_0
# WHEN MATCHED THEN 
#     UPDATE SET target.extra_col_1 = target.extra_col_1 + 10
# """
# spark.sql(query)

# end = time.time() - st
# print(f"Updated {num_rows} rows in {end:.2f} sec")
